# Zagadki

![image-2.png](attachment:image-2.png)

## Wstęp
Zagadki od wieków fascynują ludzi, pobudzając ich umysły do kreatywnego i logicznego myślenia. Od prostych łamigłówek po głębokie zagadki filozoficzne, stanowią one nie tylko formę rozrywki, ale również sztukę rozumienia języka i logicznego wnioskowania. W tym zadaniu będziesz rozwiązywał/a zagadki, polegające na odgadnięciu słowa na podstawie opisu. Wszystkie zagadki wymyślił ChatGPT (ale nie powiemy, która dokładnie wersja i w jaki sposób zachęcona), stąd niektóre mogą być trochę dziwne... Szacujemy, że ludzie są w stanie rozwiązać poprawnie trochę ponad 60% z nich. A jak dobry będzie Twój program?

## Zadanie
Napisz funckję `answer_riddle` która rozwiąże podaną na wejściu zagadkę. Rozwiązaniem jest zawsze jedno słowo. Przykładowe zagadki:

- **zagadka:** kobieta podróżująca środkiem transportu, np. samolotem, pociągiem, statkiem <br>
  **odpowiedź:** pasażerka
- **zagadka:** emocjonalne uczucie łączące dwie osoby, oparte na zaufaniu, szacunku, trosce i oddaniu<br>
  **odpowiedź:** miłość


Naszym kryterium będzie `odwrócona średnia harmoniczna` ([Mean Reciprocal Rank](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)), która działa w następujący sposób: <br>
Jeżeli na zwróconej przez Twoją funkcję liście znajdzie się prawidłowa odpowiedź, otrzymasz wówczas punkty: dokładnie $\frac{1}{k}$ punktów, gdzie $k$ jest pozycją słowa na liście. W szczególności, jeżeli Twój program zgadnie słowo (czyli umieści je na pierwszej pozycji), to otrzymasz 1 punkt. Ostatecznym kryterium jest uśredniona liczba punktów ze wszystkich zagadek.

Powyższe kryterium jest zaimplementowane poniżej przez nas.

## Ograniczenia
- Twoje finalne rozwiązanie będzie testowane w środowisku **bez** GPU.
- Twoja funkcja powinna działać na tyle szybko, aby program był w stanie udzielić odpowiedzi na 100 zagadek w maksymalnie 2 minuty bez użycia GPU.

## Dane
Dane dostępne dla Ciebie w tym zadaniu to:
* `zagadki_do_testow_clean.txt` - około 2000 przykładowych zagadek
* `plwiktionary_definitions_clean.txt` -  plik z definicjami słów wziętymi z [pl.wiktionary.org](https://pl.wiktionary.org/wiki/pl). Z wszystkich definicji z pl.wiktionary.org wzięliśmy definicje 8094 najbardziej popularnych rzeczowników (częstości liczone zgodnie z korpusem https://2018.poleval.pl/index.php/tasks#task3). Uwaga: poprawne rozwiązanie każdej zagadki znajduje się w tym pliku!

* `superbazy_clean.txt` - formy bazowe polskich słów, przygotowane na podstawie projektu https://github.com/morfologik/polimorfologik

* Wektory osadzeń słów bazowych, wytrenowane modelem Word2Vec z biblioteki Gensim, na korpusie PolEval 2018 Task3

## Uwagi i wskazówki
- Dla każdej zagadki, Twoja funkcja powinna zwrócić listę słów (co najwyżej 20), w kolejności od najbardziej (wg Twojego programu) prawdopodobnej odpowiedzi na zagadkę, do najmniej.
- Twoje rozwiazanie bedzie testowane bez dostepu do internetu

## Pliki Zgłoszeniowe
Tylko ten notebook.

## Ewaluacja
Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`. Za pomocą skryptu `validation_script.py` będziesz upewnić się, że Twoje rozwiązanie zostanie prawidłowo wykonane na naszych serwerach oceniających.

Za to zadanie możesz zdobyć pomiędzy 0 i 1.5 punktu. Zdobędziesz 0 punktów jeśli wartość kryterium `mean reciprocal rank` na zbiorze testowym wyniesie poniżej 0.02, a 1.5 punktu jeśli wyniesie powyżej 0.3. Pomiędzy tymi wartościami, wynik rośnie liniowo z wartością kryterium.

# Kod startowy

In [1]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 28.6 MB/s eta 0:00:00


In [2]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################
FINAL_EVALUATION_MODE = False
# W czasie sprawdzania Twojego rozwiązania, zmienimy tę wartość na True
# Wartość tej flagi M U S I zostać ustawiona na False w rozwiązaniu, które nam nadeślesz!

In [3]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
import nltk
from nltk.tokenize import word_tokenize as tokenize
from collections import defaultdict as dd
import math
from gensim.models import Word2Vec
import gdown
import random
import os
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import torch

## Ładowanie danych

In [4]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
path_to_data = 'data/'

bases = {}
# Dictionary mapping words to their base words
all_word_definitions = dd(list)
# Dictionary containing all base words inverse document frequency
base_idf = dd(int)

In [5]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def get_word_base(word):
    global bases
    word = word.lower()
    ret = bases.get(word)
    if ret:
        return ret
    return word

In [6]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
if not FINAL_EVALUATION_MODE:
    if not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model") \
        or not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model.syn1neg.npy") \
        or not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model.wv.vectors.npy"):
            gdown.download_folder(url="https://drive.google.com/drive/folders/1P72og_ORfL3Ojf27n-g06DT0ENduPy8C?usp=sharing", output=f"./{path_to_data}")
    nltk.download('punkt')

Retrieving folder contents


Processing file 1dCYKzdg6TihyckNM9Zxr1g9lWT1BzDXI plwiktionary_definitions_clean.txt
Processing file 1q6Ki5Y66gugM30oFcCtJj7ocMJymskSk superbazy_clean.txt
Processing file 1GZPNIR16bxFnzvbIYDLacn8TcKwiGnIh w2v_polish_lemmas.model
Processing file 1C-V5TgIAHUJp_FLD-bvks6LmMkrfQpzC w2v_polish_lemmas.model.syn1neg.npy
Processing file 1RY0Ftfx_-nPUbddYCvHq9p8yCcXbUrYS w2v_polish_lemmas.model.wv.vectors.npy
Processing file 18wrF9VyESTvIT9Z_TqUd_2wV4lP2As3t zagadki_do_testow_clean.txt


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1dCYKzdg6TihyckNM9Zxr1g9lWT1BzDXI
To: /content/data/zagadki/plwiktionary_definitions_clean.txt
100%|██████████| 1.59M/1.59M [00:00<00:00, 57.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1q6Ki5Y66gugM30oFcCtJj7ocMJymskSk
To: /content/data/zagadki/superbazy_clean.txt
100%|██████████| 43.7M/43.7M [00:02<00:00, 21.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GZPNIR16bxFnzvbIYDLacn8TcKwiGnIh
To: /content/data/zagadki/w2v_polish_lemmas.model
100%|██████████| 40.8M/40.8M [00:01<00:00, 30.7MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1C-V5TgIAHUJp_FLD-bvks6LmMkrfQpzC
From (redirected): https://drive.google.com/uc?id=1C-V5TgIAHUJp_FLD-bvks6LmMkrfQpzC&confirm=t&uuid=aa5d478f-31ee-41e3-8626-9b8336f28138
To: /content/data/zagadki/w2v_polish_lemmas.model.syn1neg.npy
100%|██████████| 958M/958M

In [7]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
for x in open(f'{path_to_data}/zagadki/superbazy_clean.txt'):
    word,base = x.lower().split()
    bases[word] = base

In [8]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
model = Word2Vec.load(f'{path_to_data}/zagadki/w2v_polish_lemmas.model')

In [9]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [10]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
for x in open(f'{path_to_data}/zagadki/plwiktionary_definitions_clean.txt'):
    word, definition = x.split('###')
    L = word.split()
    if len(L) == 1:
        word = L[0]

        definition = set(tokenize(definition.lower()))
        all_word_definitions[word].append(definition)
        for word in set(definition):
            base_idf[get_word_base(word)] += 1


for base in base_idf:
    base_idf[base] = -math.log(base_idf[base] / len(all_word_definitions))

In [11]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
answers = []
queries = []

with open(f'{path_to_data}/zagadki/zagadki_do_testow_clean.txt') as file:
    for line in file:
        line = line.replace(';;', '').split()
        answers.append(line[0])
        queries.append(tokenize(' '.join(line[1:])))

## Kod z kryteriami oceniającymi

In [12]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def mean_reciprocal_rank(real_answers, computed_answers, K=20):
    positions = []

    for real_answer, computed_answer in zip(real_answers, computed_answers):
        if real_answer in computed_answer[:K]:
            pos = computed_answer.index(real_answer) + 1
            positions.append(1/pos)

    mrr = sum(positions) / len(real_answers)
    print ('Mean Reciprocal Rank =', mrr)

    return mrr

# Twoje rozwiązanie

To jest jedyna sekcja, w której musisz coś zrobić.

In [13]:
model_name = "allegro/herbert-base-cased"
device = 'cuda'

tokenizer_herbert = AutoTokenizer.from_pretrained(model_name)
model_herbert = AutoModel.from_pretrained(model_name).to(device)

text = 'Bardzo lubię lody malinowe z bitą śmietaną.'

token_ids = tokenizer_herbert(text, return_tensors='pt')['input_ids'][0]

print ([tokenizer_herbert.decode(idx) for idx in token_ids])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/907k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/556k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['<s>', 'Bardzo', 'lubię', 'lody', 'mali', 'nowe', 'z', 'bi', 'tą', 'śmietan', 'ą', '.', '</s>']


In [14]:
model_name_papuga = 'flax-community/papuGaPT2'
tokenizer_papuga = AutoTokenizer.from_pretrained(model_name_papuga)
model_papuga = AutoModelForCausalLM.from_pretrained(model_name_papuga).to(device)

tokenizer_config.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/888k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/547k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/864 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [15]:
# data to use
# bases -  Dictionary mapping words to their base words
# all_word_definitions - word definitions
# base_idf = dd(int) - Dictionary containing all base words inverse document frequency


In [47]:
THRESHOLD_IDF = 2.89
FORBIDDEN_WORDS = ["jednak", "lub", "przecież", "ależ", "one", "poza", "bardzo", "albo", "dany", "u", "przy", "ze", "z", "do", "jako", "jak", "także"]
# oraz
# FIXED_TENSOR = torch.rand(768).to(device)

FIXED_TENSOR = torch.rand(768 + 200).to(device)


In [48]:
def tensor_to_number(tensor):
    return torch.dot(tensor, FIXED_TENSOR).item()

def find_closest_key(d, query_float):
    return min(d.keys(), key=lambda k: abs(k - query_float))

In [25]:
# torch.Tensor(model.wv[("zamek", "jak")]).shape

In [54]:
def prune_idf(l):
    return list(filter(lambda x: base_idf[bases.get(x)] > THRESHOLD_IDF and x not in FORBIDDEN_WORDS, l))

def word_to_embedding(word, tokenizer = tokenizer_herbert, m = model_herbert):

    input_ids = tokenizer(word, return_tensors='pt')['input_ids'].to(device)

    embeddings = m.embeddings.word_embeddings(input_ids)[0, :, :].mean(dim=0).detach()
    embeddings_w2v = torch.Tensor(model.wv[word]).to(device)
    return torch.cat((embeddings, embeddings_w2v), dim=0)
    # embeddings = torch.cat((embeddings, embeddings_w2v), dim=0)
    # print(embeddings.shape)
    # return embeddings

def list_of_words_to_embedding(word_list, tokenizer = tokenizer_herbert, model = model_herbert):
    input_ids = tokenizer(word_list, return_tensors='pt', padding=True)['input_ids'].to(device)

def create_embeddings(unique_words = False):
    all_embeddings = []
    d = {}
    for word, def_list in all_word_definitions.items():
        pruned_defs = list(map(prune_idf, def_list))
        new_def = [item for sublist in pruned_defs for item in sublist]
        if unique_words:
            new_def = list(set(new_def))
        try:
            embeddings = [word_to_embedding(bases.get(word)) for s in new_def]
            if len(embeddings) == 0:
                embeddings = [word_to_embedding(bases.get(word))]
            embeddings_tensor = torch.stack(embeddings)
            mean_embedding = torch.mean(embeddings_tensor, dim=0)
            all_embeddings.append(mean_embedding)
            d[tensor_to_number(mean_embedding)] = word
        except RuntimeError:
            print(word)
            print(def_list)
            print(new_def)
    return torch.stack(all_embeddings), d

In [52]:
a =word_to_embedding("konstytucja")

torch.Size([1, 3, 768])
torch.Size([3, 768])
torch.Size([768])


In [55]:
definitions_embeddings_herbert, embedding_to_word = create_embeddings()

In [56]:
definitions_embeddings_herbert.shape

torch.Size([8085, 968])

In [60]:
for k, v in embedding_to_word.items():
    new_k = find_closest_key(embedding_to_word, k)
    new_v = embedding_to_word[new_k]
    if v != new_v:
        print(v, new_v)

In [65]:
# print(embedding_to_word.get(definitions_embeddings_herbert[0]))

None


In [66]:
def k_nearest_neighbours(k, all_tensors, query_tensor):
    distances = torch.norm(all_tensors - query_tensor, dim=1)  # dim=1 computes distance row-wise
    k_neighbors = torch.topk(distances, k, largest=False)  # `largest=False` ensures smallest distances
    nearest_def_embeddings = definitions_embeddings_herbert[k_neighbors.indices]
    return [embedding_to_word[tensor_to_number(t)] for t in nearest_def_embeddings]

In [ ]:
# all_word_definitions["zamek"]
len(definitions_embeddings_herbert.shape)

2

In [77]:
def answer_riddle(riddle, K):
    # print(riddle)
    # print(random.sample(all_word_definitions.keys(), K))
    pruned_q = prune_idf(riddle)
    try:
        q_embeddings = [word_to_embedding(bases.get(s)) for s in pruned_q]
        if len(q_embeddings) == 0:
            q_embeddings = [torch.rand(768 + 200).to(device)]
        q_embeddings_tensor = torch.stack(q_embeddings)
        q_embedding = torch.mean(q_embeddings_tensor, dim=0)
    except RuntimeError:
        print(riddle)
    return k_nearest_neighbours(K, definitions_embeddings_herbert, q_embedding)

# Ewaluacja

Poniższy kod będzie służył ewaluacji rozwiązania. Po wysłaniu rozwiązania do nas zostanie wykonana funkcja `evaluate_algorithm(score_function, queries, answers, K)`, t.j. prawie identyczny kod jak niżej będzie się uruchamiał na katalogu danych `test_data` dostępnym tylko dla sprawdzających zadania.

Upewnij się przed wysłaniem, że cały notebook wykonuje się od początku do końca bez błędów i bez ingerencji użytkownika po wykonaniu polecenia `Run All`.

In [78]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def evaluate_algorithm(score_function, queries, answers, K):
    computed_answers = []
    for query in tqdm(queries, desc="queries answered"):
        computed_answers.append(score_function(set(query), K=K))
    score = mean_reciprocal_rank(answers, computed_answers, K=K)
    return score

In [79]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
# k nearest neighbours approach
if not FINAL_EVALUATION_MODE:
    # PART_OF_DATA =
    PART_OF_DATA = 293
    K = 20
    valid_queries = queries[:PART_OF_DATA]
    valid_answers = answers[:PART_OF_DATA]
    score = evaluate_algorithm(answer_riddle, valid_queries, valid_answers, K=K)
    print(f"Score: {score}")

queries answered: 100%|██████████| 293/293 [00:01<00:00, 189.63it/s]

Mean Reciprocal Rank = 0.015789457630426995
Score: 0.015789457630426995


In [80]:
if not FINAL_EVALUATION_MODE:
    # PART_OF_DATA =
    PART_OF_DATA = 1700
    K = 20
    valid_queries = queries[PART_OF_DATA:]
    valid_answers = answers[PART_OF_DATA:]
    score = evaluate_algorithm(answer_riddle, valid_queries, valid_answers, K=K)
    print(f"Score: {score}")

queries answered: 100%|██████████| 293/293 [00:01<00:00, 189.16it/s]

Mean Reciprocal Rank = 0.037122146407430494
Score: 0.037122146407430494
